In [1]:
!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d omkargurav/face-mask-dataset

 98% 160M/163M [00:01<00:00, 143MB/s]
100% 163M/163M [00:01<00:00, 127MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/face-mask-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import tensorflow
from tensorflow import keras

In [5]:
directory = '/content/data'

# Create the training dataset
train_ds = keras.preprocessing.image_dataset_from_directory(
  directory,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(150,150),
  batch_size=32)

Found 7553 files belonging to 2 classes.
Using 6043 files for training.


In [6]:
# Create the validation dataset
val_ds = keras.preprocessing.image_dataset_from_directory(
  directory,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(150,150),
  batch_size=32)

Found 7553 files belonging to 2 classes.
Using 1510 files for validation.


In [7]:
classes = train_ds.class_names

In [8]:
classes

['with_mask', 'without_mask']

In [9]:
def process(image, label):
  image = tensorflow.cast(image/255, tensorflow.float32)
  return image, label

In [10]:
training_data = train_ds.map(process)
validation_data = val_ds.map(process)

In [11]:
from keras.applications.vgg16 import VGG16

In [12]:
conv_base  = VGG16(weights='imagenet', include_top=False ,input_shape=(150,150,3))

58889256/58889256 [==============================] - 0s 0us/step


In [13]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [14]:
from keras.models import Sequential
from keras.layers import *

In [15]:
conv_base.trainable = False

In [16]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())

model.add(Dense(4096, activation = 'relu'))
model.add(Dense(4096, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 4096)              33558528  
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 1)                 4097      
                                                                 
Total params: 65058625 (248.18 MB)
Trainable params: 50343937 (192.05 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [18]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
model.fit(training_data, epochs = 5, validation_data= validation_data)

Epoch 1/5
189/189 [==============================] - 31s 123ms/step - loss: 0.4112 - accuracy: 0.9303 - val_loss: 0.0781 - val_accuracy: 0.9709
Epoch 2/5
189/189 [==============================] - 21s 106ms/step - loss: 0.0566 - accuracy: 0.9793 - val_loss: 0.1365 - val_accuracy: 0.9530
Epoch 3/5
189/189 [==============================] - 21s 107ms/step - loss: 0.0398 - accuracy: 0.9851 - val_loss: 0.0746 - val_accuracy: 0.9748
Epoch 4/5
189/189 [==============================] - 19s 100ms/step - loss: 0.0213 - accuracy: 0.9924 - val_loss: 0.0744 - val_accuracy: 0.9722
Epoch 5/5
189/189 [==============================] - 21s 109ms/step - loss: 0.0164 - accuracy: 0.9934 - val_loss: 0.1134 - val_accuracy: 0.9695


In [20]:
import numpy as np
from keras.preprocessing import image

In [21]:
img_path = '/content/data/with_mask/with_mask_1032.jpg'

In [22]:
def prediction(path, model, class_names):
  img = image.load_img(path, target_size=(150,150))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = img/255

  y_prob = model.predict(img)[0][0]
  print(f'Probability: {y_prob}')

  # Threshold the probability to get class
  class_idx = (y_prob > 0.5).astype('int')
  class_name = class_names[class_idx]

  print(f'Class: {class_name}')


In [23]:
prediction(img_path, model, classes)

1/1 [==============================] - 1s 1s/step
Probability: 8.672448870150617e-10
Class: with_mask


In [24]:
from keras.models import load_model

In [25]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
hmodel = load_model('model.h5')

In [44]:
prediction(img_path, hmodel, classes)

1/1 [==============================] - 0s 263ms/step
Probability: 7.576098637329665e-10
Class: with_mask
